# REDCap Surveys
The REDCap surveys have a wealth of information that we can use to compare to some of the IAQ conditions -- perhaps there is a link!

In [1]:
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

# Processing 
In this notebook, we work on processing the home environment surveys.

In [9]:
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import seaborn as sns
import matplotlib.dates as mdates

# Data Import and Summary
There are a variety of questions -- some of which aren't relevant to my research -- that we can look at.

In [10]:
ee = pd.read_csv("../data/raw/utx000/surveys/EESurvey_E1_labels.csv",parse_dates=["Survey Timestamp"],index_col=0)
ee.dropna(subset=["Survey Timestamp"], axis="rows",inplace=True)
ee.drop([column for column in ee.columns if "gender" in column.lower() or "family" in column.lower()],axis="columns",inplace=True)
ee.replace("Unchecked",0,inplace=True)
ee.replace("Checked",1,inplace=True)
ee.head()

,Event Name,Survey Timestamp,"In the past week, have you traveled to a different city?",What type of building are you currently living in? (choice=Apartment),What type of building are you currently living in? (choice=Duplex),What type of building are you currently living in? (choice=Stand-alone House),What type of building are you currently living in? (choice=Dormitory),What type of building are you currently living in? (choice=Motel/Hotel),What type of building are you currently living in? (choice=Other),Are you living with the same people as last week?,"In the past week, have you opened the windows in your residence?",Why did you open your windows? (choice=To change indoor temperature),Why did you open your windows? (choice=To get fresh air),Why did you open your windows? (choice=To change temperature and get fresh air),Why did you open your windows? (choice=Other),"In the past week, have you tried to isolate yourself in some part of your home?",Why? (choice=To be able to work without interruption),Why? (choice=To have more privacy),Why? (choice=To isolate from a sick member of the household),Why? (choice=To isolate myself because I am not feeling well),Why? (choice=Other),How did you try to isolate yourself? (choice=Stayed in bedroom),How did you try to isolate yourself? (choice=Stayed in another room of the house),How did you try to isolate yourself? (choice=Closed doors between rooms),How did you try to isolate yourself? (choice=Closed windows in room),How did you try to isolate yourself? (choice=Tried to change ventilation in the isolated space),How did you try to isolate yourself? (choice=Minimized contact with other household members),How did you try to isolate yourself? (choice=Other),"Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Stagnant)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Smelly)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Earthy)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Moldy)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Cooking)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Fragrant)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Fresh (well ventilated))","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Obnoxious)","Based on your sense of smell, how can you describe the 'freshness' of air at your residence? (choice=Other)","In the past week, have you changed your air conditioning filter?",Are you currently using a portable air cleaner at home?,Did you use the air cleaner prior to the COVID-19 outbreak?,"In the past week, has your house been uncomfortably hot?",Can you easily control the temperature within your home?,"In the past week, has your home been uncomfortably humid (sticky)?","In the past week, have the hard surfaces in your residence been cleaned?",What hard surfaces were cleaned? (choice=Kitchen Counters),What hard surfaces were cleaned? (choice=Floors),What hard surfaces were cleaned? (choice=Door Knobs),What hard surfaces were cleaned? (choice=Table Tops),What hard surfaces were cleaned? (choice=Sinks),What hard surfaces were cleaned? (choice=Toilets),What hard surfaces were cleaned? (choice=Appliance Knobs and Handles),"In the past week, has your home been vacuumed?","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Bleach)","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Ammonia)","In the past week, have any of the following cleaners been used in your home (Check all that apply )? (choice=Pinesol)","In the past week, hav

From the raw data, we can see the following subcategories:
1. Building Type
3. Window Opening
4. Isolation
5. Smell
6. Cleaners Used and Location
7. Allergies
8. COVID

## Common Functions
There are some functions that can be used across variable types. They are below with a docstring indicating their use.

In [31]:
def check_resonse(row):
    """
    Checks to see if the user responded with anything (given by 1s). Returns 1 indicating the user did NOT respond, else 0.
    """
    s = sum(row)
    if s == 0:
        return 1
    else:
        return 0

## Building Type 
The building types are separated by checked and unchecked boxes. Here we summarize the building type into one column `building_type`

In [21]:
def get_building_type(row):
    for i, building_type in enumerate(["apartment","duplex","house","dormitory","hotel","other"]):
        if row.iloc[i] == 1:
            return building_type
        

In [22]:
ee_building_type = ee[[column for column in ee.columns if "What type of building" in column]]
ee_building_type["building_type"] = ee_building_type.apply(get_building_type, axis="columns")
ee_building_type.drop([column for column in ee_building_type.columns if "What type of building" in column], axis="columns", inplace=True)
ee_building_type.head()

,building_type
Record ID,
1,house
4,apartment
5,apartment
6,house
9,apartment


## Window

In [23]:
def get_window_use(row):
    d = {"use":[]}
    if row.iloc[0] != "No":
        uses = []
        for i, choice in enumerate(["change_temperature","fresh_air","both","other"]):
            if row.iloc[i+1] == 1:
                uses.append(choice)
                
        d["use"] = uses
        
    return d 

<div class="alert alert-block alert-danger">
    
Above functions is `deprecated`
    
</div>

In [27]:
ee_window = ee[[column for column in ee.columns if "your windows" in column.lower()]]
#ee_window["window_use"] = ee_window.apply(get_window_use, axis="columns")
#ee_window.drop([column for column in ee_window.columns if "windows" in column.lower()],axis="columns",inplace=True)
ee_window.columns = ["change_temperature","fresh_air","both","other"]
ee_window["no_window_use"] = ee_window.apply(check_resonse, axis="columns")
ee_window.head()

,change_temperature,fresh_air,both,other,no_window_use
Record ID,,,,,
1,0,1,0,0,0
4,0,0,0,0,1
5,0,0,0,0,1
6,0,0,0,0,1
9,0,1,0,0,0


## Smell
The first thing we want to check is if people check multiple options for the smell category to decided whether or not we should keep each column or separate everything out.

In [20]:
def check_smells(row):
    s = sum(row)
    if s == 0:
        return 1
    else:
        return s

In [21]:
smell_df = ee[[column for column in ee.columns if "sense of smell" in column]]
smell_df["n_smells"] = smell_df.apply(check_smells,axis="columns")
smell_df["n_smells"].value_counts()

1    37
2    15
3     3
Name: n_smells, dtype: int64

<div class="alert alert-block alert-warning">
    
Looks like we need to keep each of the smell columns for one-hot encoding
    
</div>

In [9]:
def get_smell(row):
    for i, smell_type in enumerate(["stagnant","smelly","earthy","moldy","cooking","fragrant","well_ventilated","obnoxious","other"]):
        if row.iloc[i] == 1:
            return smell_type

<div class="alert alert-block alert-danger">
    
Above functions is `deprecated`
    
</div>

In [29]:
ee_smell = ee[[column for column in ee.columns if "smell" in column.lower()]]
#ee_smell["smell_type"] = ee_smell.apply(get_smell,axis="columns")
#ee_smell.drop([column for column in ee_smell.columns if "smell" in column.lower() and "type" not in column.lower()],axis="columns",inplace=True)
ee_smell.columns = ["stagnant","smelly","earthy","moldy","cooking","fragrant","well_ventilated","obnoxious","other"]
ee_smell["no_smell"] = ee_smell.apply(check_resonse, axis="columns")
ee_smell.describe()

,stagnant,smelly,earthy,moldy,cooking,fragrant,well_ventilated,obnoxious,other,no_smell
count,55.000000,55.000000,55.000000,55.0,55.000000,55.000000,55.000000,55.0,55.000000,55.0
mean,0.272727,0.036364,0.054545,0.0,0.327273,0.036364,0.600000,0.0,0.054545,0.0
std,0.449467,0.188919,0.229184,0.0,0.473542,0.188919,0.494413,0.0,0.229184,0.0
min,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
25%,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.0
75%,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,1.000000,0.0,0.000000,0.0
max,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,0.0


In [30]:
ee_smell["no_smell"].value_counts()

0    55
Name: no_smell, dtype: int64

<div class="alert alert-block alert-success">
    
Every participant responded with a smell, but we should still include the `no_smell` column for completeness-sake
    
</div>

## Allergy

In [11]:
def get_allergy(row):
    if row.iloc[0] == "No":
        return 0
    else:
        for i, allergy_intensity in enumerate(["significantly_worse","somewhat_worse","same","somewhat_better","significantly_better"]):
            if row.iloc[i] == 1:
                return allergy_intensity
            
    return 0

In [12]:
ee_allergy = ee[[column for column in ee.columns if "allergy" in column.lower()]]
ee_allergy["allergy_intensity"] = ee_allergy.apply(get_allergy,axis="columns")
ee_allergy.drop([column for column in ee_allergy.columns if "allergy" in column.lower() and "intensity" not in column.lower()],axis="columns",inplace=True)
ee_allergy.head()

,allergy_intensity
Record ID,
1,same
4,somewhat_better
5,0
6,somewhat_better
9,same


## Cleaner Use

In [13]:
def get_cleaner_use(row):
    """
    
    """
    d = {"cleaners":[]}
    for i, cleaner in enumerate(["bleach","ammonia","pinesol","vinegar","alcohol","disinfectant_wipes","soap_and_water","floor_cleaners"]):
        cleaners = []
        if row.iloc[i] == 1:
            cleaners.append(cleaner)
        
    d["cleaners"] = cleaners
        
    return d

<div class="alert alert-block alert-danger">
    
Above functions is `deprecated`
    
</div>

In [15]:
ee_cleaner_use = ee[[column for column in ee.columns if "cleaners been used" in column.lower()]]
ee_cleaner_use.columns = ["bleach","ammonia","pinesol","vinegar","alcohol","disinfectant_wipes","soap_and_water","floor_cleaners"]
ee_cleaner_use.head()

,bleach,ammonia,pinesol,vinegar,alcohol,disinfectant_wipes,soap_and_water,floor_cleaners
Record ID,,,,,,,,
1,0,0,0,0,0,1,1,0
4,1,0,1,0,0,1,1,0
5,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,1,1
9,0,0,0,0,0,1,1,0


## Cleaner Location

In [14]:
def get_cleaner_location(row):
    """
    
    """
    d = {"bleach":[],"ammonia":[],"pinesol":[],"vinegar":[],"alcohol":[],"disinfectant_wipes":[],"soap_and_water":[],"floor_cleaners":[]}
    for cleaner in d.keys():
        temp = row[[column for column in row.index if cleaner in column.replace(" ","_").lower()]]
        locs = []

        for i in range(len(temp)):
            if temp.iloc[i] == 1:
                locs.append(temp.index[i].split("=")[1][:-1])
                
        d[cleaner] = (locs)
        
    return d

In [17]:
ee_cleaner_locs = ee[[column for column in ee.columns if "in which rooms" in column.lower()]]
ee_cleaner_locs["cleaner_locations"] = ee_cleaner_locs.apply(get_cleaner_location,axis="columns")
ee_cleaner_locs.drop([column for column in ee.columns if "in which rooms" in column.lower()],axis="columns",inplace=True)
ee_cleaner_locs.head()

,cleaner_locations
Record ID,
1,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
4,"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':..."
5,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
6,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
9,"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."


# Combining

In [45]:
data = [ee_building_type,ee_window,ee_smell,ee_allergy,ee_cleaner_use,ee_cleaner_locs]
for df in data:
    print("Observations: ",len(df))
    df.sort_index(inplace=True)
df = pd.concat(data,axis="columns")
df.index.rename("redcap",inplace=True)
df.head()

Observations:  55
Observations:  55
Observations:  55
Observations:  55
Observations:  55
Observations:  55


,building_type,window_use,smell_type,allergy_intensity,cleaner,cleaner_locations
redcap,,,,,,
1,house,{'use': ['fresh_air']},well_ventilated,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
4,apartment,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':..."
5,apartment,{'use': []},smelly,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
6,house,{'use': []},well_ventilated,somewhat_better,{'cleaners': ['floor_cleaners']},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."
9,apartment,{'use': ['fresh_air']},cooking,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '..."


In [53]:
id_crossover = pd.read_excel(f'../data/raw/utx000/admin/id_crossover.xlsx',sheet_name='all')
df_final = df.merge(right=id_crossover,left_index=True,right_on="redcap").drop(["first","last"],axis="columns")
df_final.head()

,building_type,window_use,smell_type,allergy_intensity,cleaner,cleaner_locations,redcap,beiwe,beacon,fitbit
1,house,{'use': ['fresh_air']},well_ventilated,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",1,y1tvkx14,NaN,18
3,apartment,{'use': []},stagnant,somewhat_better,{'cleaners': []},"{'bleach': ['Kitchen', 'Bathroom'], 'ammonia':...",4,idbkjh8u,30.0,22
4,apartment,{'use': []},smelly,0,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",5,eyf8oqwl,NaN,23
5,house,{'use': []},well_ventilated,somewhat_better,{'cleaners': ['floor_cleaners']},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",6,789gcb6u,NaN,17
7,apartment,{'use': ['fresh_air']},cooking,same,{'cleaners': []},"{'bleach': [], 'ammonia': [], 'pinesol': [], '...",9,fxw5xupi,NaN,NaN


---